In [1]:
import pip

In [2]:
pip.main(['install','/datasets/home/34/234/cs291gbl/fastText/. --user'])

Invalid requirement: '/datasets/home/34/234/cs291gbl/fastText/. --user'
It looks like a path. Does it exist ?
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


1

In [3]:
import numpy as np
import pip
import torch
from torch import nn
import fastText
import argparse
from torch.autograd import Variable
import io
# import torch
print(torch.__version__)

0.3.1.post3


In [36]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--dis_hidden_dim', type=int, default=2048, help='dimension of hidden state')
# parser.add_argument('--dis_dropout',type=float, default=0.5, help='dropout value')
# parser.add_argument('--dis_input_dropout', type=float, default=0.3, help='Point Number [default: 1024]')

# params=parser.parse_args()
def read_txt_embeddings(path, full_vocab=False):
    """
    Reload pretrained embeddings from a text file.
    """
    word2id = {}
    vectors = []
    id2word={}

    # load pretrained embeddings
#     lang = params.src_lang if source else params.tgt_lang
    emb_path =path
    _emb_dim_file = 300
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for i, line in enumerate(f):
            if i == 0:
                split = line.split()
                assert len(split) == 2
                assert _emb_dim_file == int(split[1])
            else:
                word, vect = line.rstrip().split(' ', 1)
                if not full_vocab:
                    word = word.lower()
                vect = np.fromstring(vect, sep=' ')
                if np.linalg.norm(vect) == 0:  # avoid to have null embeddings
                    vect[0] = 0.01
                if word in word2id:
                    if full_vocab:
                        logger.warning("Word '%s' found twice in %s embedding file"
                                       % (word, 'source' if source else 'target'))
                else:
                    if not vect.shape == (_emb_dim_file,):
                        logger.warning("Invalid dimension (%i) for %s word '%s' in line %i."
                                       % (vect.shape[0], 'source' if source else 'target', word, i))
                        continue
                    assert vect.shape == (_emb_dim_file,), i
                    word2id[word] = len(word2id)
                    id2word[len(word2id)]=word
                    vectors.append(vect[None])
            if len(word2id) >= 200000 and not full_vocab:
                break

    assert len(word2id) == len(vectors)
    print("Loaded %i pre-trained word embeddings." % len(vectors))

    # compute new vocabulary / embeddings
    id2word = {v: k for k, v in word2id.items()}
#     dico = Dictionary(id2word, word2id, lang)
    embeddings = np.concatenate(vectors, 0)
    embeddings = torch.from_numpy(embeddings).float()
    embeddings = embeddings
    assert embeddings.size() == (200000, 300)
    return embeddings,word2id,id2word

# def load_vec(emb_path, nmax=50000):
#     vectors = []
#     word2id = {}
#     with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
#         next(f)
#         for i, line in enumerate(f):
#             word, vect = line.rstrip().split(' ', 1)
#             vect = np.fromstring(vect, sep=' ')
#             assert word not in word2id, 'word found twice'
#             vectors.append(vect)
#             word2id[word] = len(word2id)
#             if len(word2id) == nmax:
#                 break
#     id2word = {v: k for k, v in word2id.items()}
#     embeddings = np.vstack(vectors)
#     return embeddings, id2word, word2id

def load_fasttext_model(path):
    """
    Load a binarized fastText model.
    """
    try:
        import fastText
    except ImportError:
        raise Exception("Unable to import fastText. Please install fastText for Python: "
                        "https://github.com/facebookresearch/fastText")
    return fastText.load_model(path)

In [5]:
# english.get
# src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
# tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

In [37]:

# english=load_fasttext_model("data/wiki.en.bin")
# spanish=load_fasttext_model("data/wiki.es.bin")
src_embeddings,src_word2id,src_id2word=read_txt_embeddings("data/wiki.en.vec")
tgt_embeddings,tgt_word2id,tgt_id2word=read_txt_embeddings("data/wiki.es.vec")

Loaded 200000 pre-trained word embeddings.
Loaded 200000 pre-trained word embeddings.


In [7]:
# words = english.get_labels()
# english_embeddings = torch.from_numpy(np.concatenate([english.get_word_vector(w)[None] for w in words], 0))[:200000]
# spanish_words = spanish.get_labels()
# spanish_embeddings = torch.from_numpy(np.concatenate([spanish.get_word_vector(w)[None] for w in Swords], 0))[:200000]
def get_batch2(X,M):
    '''
    get batch for pytorch model
    '''
    # TODO fix and make it nicer, there is pytorch forum question
    #X,Y = X.data.numpy(), Y.data.numpy()
#     X,Y = X, Y
    N = X.size()[0]
    N=200000
    batch_indices = torch.LongTensor( np.random.randint(0,N,size=M) )
#     print(batch_indices)
    batch_xs = torch.index_select(X,0,batch_indices)
#     batch_ys = torch.index_select(Y,0,batch_indices)
    return Variable(batch_xs, requires_grad=False).cuda()

In [8]:
class Discriminator(nn.Module):

    def __init__(self, dis_hid_dim,dis_dropout,dis_input_dropout):
        super(Discriminator, self).__init__()

        self.emb_dim = 300
        self.dis_layers = 3
        self.dis_hid_dim = dis_hid_dim
        self.dis_dropout = dis_dropout
        self.dis_input_dropout = dis_input_dropout

        layers = [nn.Dropout(self.dis_input_dropout)]
        for i in range(self.dis_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.dis_hid_dim
            output_dim = 1 if i == self.dis_layers else self.dis_hid_dim
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.dis_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.dis_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        assert x.dim() == 2 and x.size(1) == self.emb_dim
        return self.layers(x).view(-1)

In [9]:

def Orthogonal(mapping,beta):
    W=mapping.weight.data
#     print(type(W))
  
    mapping.weight.data.copy_((1 + beta) * W - beta * W.mm(W.transpose(0, 1).mm(W)))
#     print(mapping.weight.data.transpose(0, 1).mm(W))
    
    return mapping


In [10]:
#set up linear mapping,
mapping = nn.Linear(300, 300, bias=False)
mapping.weight.data.copy_(torch.eye(300))
modelDisc= Discriminator(2048,0.3,0.1)
mapping.cuda()
modelDisc.cuda()

Discriminator(
  (layers): Sequential(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=300, out_features=2048, bias=True)
    (2): LeakyReLU(0.2)
    (3): Dropout(p=0.3)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): LeakyReLU(0.2)
    (6): Dropout(p=0.3)
    (7): Linear(in_features=2048, out_features=2048, bias=True)
    (8): LeakyReLU(0.2)
    (9): Dropout(p=0.3)
    (10): Linear(in_features=2048, out_features=1, bias=True)
    (11): Sigmoid()
  )
)

In [11]:
linoptimizer = torch.optim.SGD(mapping.parameters(), lr=0.1)
discoptimizer= torch.optim.SGD(modelDisc.parameters(), lr=0.1)
# torch.optim.
loss_fn=torch.nn.BCELoss()
# print(type(tgt_embeddings))
def get_xy():
#get minibatch of spanish words
    spanish_batch=get_batch2(tgt_embeddings,batch_size)
    #getminibatch of english words
    english_batch=get_batch2(src_embeddings,batch_size)

    #generate fake spanish embeddings
    s_fake= mapping(english_batch)

    #stack real and fake
    x = torch.cat([spanish_batch, s_fake], 0)
    #         print(x.shape)
    y = torch.FloatTensor(2 * batch_size).zero_()
    
    y[batch_size:] = 0.05
    y[:batch_size]=1-0.05
    y=Variable(y,requires_grad=False).cuda()
    return x,y

In [12]:
epoch=100000
batch_size=32
# E_batch=Batch(english_embeddings)
# S_batch=Batch(spanish_embeddings)
dog=Variable(src_embeddings[src_word2id['dog']]).cuda()

perro=Variable(tgt_embeddings[tgt_word2id['perro']]).cuda()
for _ in range(epoch):
      
    
    modelDisc.train()
    mapping.eval()
    for __ in range(5):
        
        x,y=get_xy()
        y_pred = modelDisc(x)
        # Compute and print loss.
        

        loss= loss_fn(y_pred, y)

        discoptimizer.zero_grad()
       
        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()
#         print(loss.data)

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        discoptimizer.step()
        
        
    for __ in range(3):
        modelDisc.eval()
        mapping.train()
        x,y=get_xy()
        y=1-y
        y_pred = modelDisc(x)
        loss2= loss_fn(y_pred, y)
        linoptimizer.zero_grad()
        loss2.backward()
        linoptimizer.step()
        
        if _%1:
            mapping=Orthogonal(mapping,0.001)
        cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    #     print(type(perro),type(mapping(dog).cpu()))
    print(cos(perro.cpu(),mapping(dog).cpu()).data)
    
    
    


1.00000e-02 *
  6.3982
[torch.FloatTensor of size 1]


1.00000e-02 *
  6.4470
[torch.FloatTensor of size 1]


1.00000e-02 *
  6.5307
[torch.FloatTensor of size 1]


1.00000e-02 *
  6.6647
[torch.FloatTensor of size 1]


1.00000e-02 *
  6.8615
[torch.FloatTensor of size 1]


1.00000e-02 *
  7.1775
[torch.FloatTensor of size 1]


1.00000e-02 *
  7.7003
[torch.FloatTensor of size 1]


1.00000e-02 *
  8.5207
[torch.FloatTensor of size 1]


1.00000e-02 *
  9.8992
[torch.FloatTensor of size 1]


 0.1180
[torch.FloatTensor of size 1]


 0.1395
[torch.FloatTensor of size 1]


 0.1597
[torch.FloatTensor of size 1]


 0.1739
[torch.FloatTensor of size 1]


 0.1833
[torch.FloatTensor of size 1]


 0.1902
[torch.FloatTensor of size 1]


 0.1960
[torch.FloatTensor of size 1]


 0.2007
[torch.FloatTensor of size 1]


 0.2044
[torch.FloatTensor of size 1]


 0.2067
[torch.FloatTensor of size 1]


 0.2087
[torch.FloatTensor of size 1]


 0.2102
[torch.FloatTensor of size 1]


 0.2108
[torch.FloatTens


 0.1960
[torch.FloatTensor of size 1]


 0.1963
[torch.FloatTensor of size 1]


 0.1976
[torch.FloatTensor of size 1]


 0.1986
[torch.FloatTensor of size 1]


 0.1937
[torch.FloatTensor of size 1]


 0.1956
[torch.FloatTensor of size 1]


 0.2012
[torch.FloatTensor of size 1]


 0.2033
[torch.FloatTensor of size 1]


 0.2041
[torch.FloatTensor of size 1]


 0.2063
[torch.FloatTensor of size 1]


 0.2046
[torch.FloatTensor of size 1]


 0.2034
[torch.FloatTensor of size 1]


 0.2057
[torch.FloatTensor of size 1]


 0.2066
[torch.FloatTensor of size 1]


 0.2048
[torch.FloatTensor of size 1]


 0.1998
[torch.FloatTensor of size 1]


 0.1975
[torch.FloatTensor of size 1]


 0.1960
[torch.FloatTensor of size 1]


 0.1920
[torch.FloatTensor of size 1]


 0.1884
[torch.FloatTensor of size 1]


 0.1903
[torch.FloatTensor of size 1]


 0.1946
[torch.FloatTensor of size 1]


 0.1985
[torch.FloatTensor of size 1]


 0.2053
[torch.FloatTensor of size 1]


 0.2107
[torch.FloatTensor of size 1]




 0.2178
[torch.FloatTensor of size 1]


 0.2250
[torch.FloatTensor of size 1]


 0.2273
[torch.FloatTensor of size 1]


 0.2340
[torch.FloatTensor of size 1]


 0.2406
[torch.FloatTensor of size 1]


 0.2417
[torch.FloatTensor of size 1]


 0.2485
[torch.FloatTensor of size 1]


 0.2480
[torch.FloatTensor of size 1]


 0.2509
[torch.FloatTensor of size 1]


 0.2477
[torch.FloatTensor of size 1]


 0.2360
[torch.FloatTensor of size 1]


 0.2275
[torch.FloatTensor of size 1]


 0.2228
[torch.FloatTensor of size 1]


 0.2246
[torch.FloatTensor of size 1]


 0.2279
[torch.FloatTensor of size 1]


 0.2376
[torch.FloatTensor of size 1]


 0.2488
[torch.FloatTensor of size 1]


 0.2499
[torch.FloatTensor of size 1]


 0.2426
[torch.FloatTensor of size 1]


 0.2332
[torch.FloatTensor of size 1]


 0.2228
[torch.FloatTensor of size 1]


 0.2133
[torch.FloatTensor of size 1]


 0.2142
[torch.FloatTensor of size 1]


 0.2213
[torch.FloatTensor of size 1]


 0.2303
[torch.FloatTensor of size 1]




 0.2318
[torch.FloatTensor of size 1]


 0.2393
[torch.FloatTensor of size 1]


 0.2434
[torch.FloatTensor of size 1]


 0.2315
[torch.FloatTensor of size 1]


 0.2318
[torch.FloatTensor of size 1]


 0.2279
[torch.FloatTensor of size 1]


 0.2213
[torch.FloatTensor of size 1]


 0.2158
[torch.FloatTensor of size 1]


 0.2035
[torch.FloatTensor of size 1]


 0.2054
[torch.FloatTensor of size 1]


 0.2133
[torch.FloatTensor of size 1]


 0.2249
[torch.FloatTensor of size 1]


 0.2302
[torch.FloatTensor of size 1]


 0.2312
[torch.FloatTensor of size 1]


 0.2307
[torch.FloatTensor of size 1]


 0.2269
[torch.FloatTensor of size 1]


 0.2250
[torch.FloatTensor of size 1]


 0.2239
[torch.FloatTensor of size 1]


 0.2165
[torch.FloatTensor of size 1]


 0.2130
[torch.FloatTensor of size 1]


 0.2063
[torch.FloatTensor of size 1]


 0.2120
[torch.FloatTensor of size 1]


 0.2175
[torch.FloatTensor of size 1]


 0.2237
[torch.FloatTensor of size 1]


 0.2160
[torch.FloatTensor of size 1]




 0.2730
[torch.FloatTensor of size 1]


 0.2794
[torch.FloatTensor of size 1]


 0.2890
[torch.FloatTensor of size 1]


 0.2839
[torch.FloatTensor of size 1]


 0.2812
[torch.FloatTensor of size 1]


 0.2721
[torch.FloatTensor of size 1]


 0.2556
[torch.FloatTensor of size 1]


 0.2485
[torch.FloatTensor of size 1]


 0.2434
[torch.FloatTensor of size 1]


 0.2497
[torch.FloatTensor of size 1]


 0.2631
[torch.FloatTensor of size 1]


 0.2736
[torch.FloatTensor of size 1]


 0.2834
[torch.FloatTensor of size 1]


 0.2868
[torch.FloatTensor of size 1]


 0.2884
[torch.FloatTensor of size 1]


 0.2852
[torch.FloatTensor of size 1]


 0.2730
[torch.FloatTensor of size 1]


 0.2554
[torch.FloatTensor of size 1]


 0.2497
[torch.FloatTensor of size 1]


 0.2500
[torch.FloatTensor of size 1]


 0.2607
[torch.FloatTensor of size 1]


 0.2639
[torch.FloatTensor of size 1]


 0.2712
[torch.FloatTensor of size 1]


 0.2722
[torch.FloatTensor of size 1]


 0.2642
[torch.FloatTensor of size 1]



...


 0.6274
[torch.FloatTensor of size 1]


 0.6296
[torch.FloatTensor of size 1]


 0.6347
[torch.FloatTensor of size 1]


 0.6422
[torch.FloatTensor of size 1]


 0.6480
[torch.FloatTensor of size 1]


 0.6503
[torch.FloatTensor of size 1]


 0.6468
[torch.FloatTensor of size 1]


 0.6435
[torch.FloatTensor of size 1]


 0.6400
[torch.FloatTensor of size 1]


 0.6353
[torch.FloatTensor of size 1]


 0.6296
[torch.FloatTensor of size 1]


 0.6267
[torch.FloatTensor of size 1]


 0.6277
[torch.FloatTensor of size 1]


 0.6307
[torch.FloatTensor of size 1]


 0.6364
[torch.FloatTensor of size 1]


 0.6418
[torch.FloatTensor of size 1]


 0.6428
[torch.FloatTensor of size 1]


 0.6424
[torch.FloatTensor of size 1]


 0.6398
[torch.FloatTensor of size 1]


 0.6388
[torch.FloatTensor of size 1]


 0.6369
[torch.FloatTensor of size 1]


 0.6357
[torch.FloatTensor of size 1]


 0.6337
[torch.FloatTensor of size 1]


 0.6301
[torch.FloatTensor of size 1]


 0.6291
[torch.FloatTensor of size 1]




 0.6301
[torch.FloatTensor of size 1]


 0.6304
[torch.FloatTensor of size 1]


 0.6360
[torch.FloatTensor of size 1]


 0.6373
[torch.FloatTensor of size 1]


 0.6400
[torch.FloatTensor of size 1]


 0.6475
[torch.FloatTensor of size 1]


 0.6487
[torch.FloatTensor of size 1]


 0.6515
[torch.FloatTensor of size 1]


 0.6537
[torch.FloatTensor of size 1]


 0.6559
[torch.FloatTensor of size 1]


 0.6532
[torch.FloatTensor of size 1]


 0.6509
[torch.FloatTensor of size 1]


 0.6517
[torch.FloatTensor of size 1]


 0.6565
[torch.FloatTensor of size 1]


 0.6581
[torch.FloatTensor of size 1]


 0.6570
[torch.FloatTensor of size 1]


 0.6555
[torch.FloatTensor of size 1]


 0.6537
[torch.FloatTensor of size 1]


 0.6497
[torch.FloatTensor of size 1]


 0.6497
[torch.FloatTensor of size 1]


 0.6509
[torch.FloatTensor of size 1]


 0.6472
[torch.FloatTensor of size 1]


 0.6440
[torch.FloatTensor of size 1]


 0.6407
[torch.FloatTensor of size 1]


 0.6390
[torch.FloatTensor of size 1]




 0.6223
[torch.FloatTensor of size 1]


 0.6243
[torch.FloatTensor of size 1]


 0.6241
[torch.FloatTensor of size 1]


 0.6283
[torch.FloatTensor of size 1]


 0.6312
[torch.FloatTensor of size 1]


 0.6348
[torch.FloatTensor of size 1]


 0.6353
[torch.FloatTensor of size 1]


 0.6333
[torch.FloatTensor of size 1]


 0.6345
[torch.FloatTensor of size 1]


 0.6357
[torch.FloatTensor of size 1]


 0.6375
[torch.FloatTensor of size 1]


 0.6417
[torch.FloatTensor of size 1]


 0.6415
[torch.FloatTensor of size 1]


 0.6403
[torch.FloatTensor of size 1]


 0.6437
[torch.FloatTensor of size 1]


 0.6467
[torch.FloatTensor of size 1]


 0.6496
[torch.FloatTensor of size 1]


 0.6489
[torch.FloatTensor of size 1]


 0.6488
[torch.FloatTensor of size 1]


 0.6477
[torch.FloatTensor of size 1]


 0.6452
[torch.FloatTensor of size 1]


 0.6398
[torch.FloatTensor of size 1]


 0.6367
[torch.FloatTensor of size 1]


 0.6367
[torch.FloatTensor of size 1]


 0.6407
[torch.FloatTensor of size 1]




 0.6370
[torch.FloatTensor of size 1]


 0.6368
[torch.FloatTensor of size 1]


 0.6312
[torch.FloatTensor of size 1]


 0.6246
[torch.FloatTensor of size 1]


 0.6161
[torch.FloatTensor of size 1]


 0.6146
[torch.FloatTensor of size 1]


 0.6161
[torch.FloatTensor of size 1]


 0.6133
[torch.FloatTensor of size 1]


 0.6178
[torch.FloatTensor of size 1]


 0.6202
[torch.FloatTensor of size 1]


 0.6278
[torch.FloatTensor of size 1]


 0.6276
[torch.FloatTensor of size 1]


 0.6290
[torch.FloatTensor of size 1]


 0.6307
[torch.FloatTensor of size 1]


 0.6253
[torch.FloatTensor of size 1]


 0.6276
[torch.FloatTensor of size 1]


 0.6294
[torch.FloatTensor of size 1]


 0.6294
[torch.FloatTensor of size 1]


 0.6310
[torch.FloatTensor of size 1]


 0.6308
[torch.FloatTensor of size 1]


 0.6277
[torch.FloatTensor of size 1]


 0.6271
[torch.FloatTensor of size 1]


 0.6246
[torch.FloatTensor of size 1]


 0.6255
[torch.FloatTensor of size 1]


 0.6242
[torch.FloatTensor of size 1]



KeyboardInterrupt: 

In [35]:
# # spanish_embeddings
tgt_emb=tgt_embeddings.cpu().numpy()
# # print(tgt_emd.shape)


def get_nn(word_emb, tgt_emb, tgt_id2word, K=5):
  
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [ ]:
# def create_map(words,language):
#     id2word={}
#     for word in words:
#         id=language.get_word_id(word)
#         id2word[id]=word
#     return id2word
# spanish_map=create_map(Swords,spanish)
        

In [45]:
cat_embedding=mapping(Variable(src_embeddings[src_word2id['same']],requires_grad=False).cuda()).cpu().data
word_emd=cat_embedding.numpy()
get_nn(word_emd,tgt_emb,tgt_id2word)


0.5993 - exactamente
0.5965 - misma
0.5547 - diferente
0.5530 - aceptablemente
0.5523 - independientemente


In [46]:
cat_embedding=tgt_embeddings[tgt_word2id['misma']]
word_emd=cat_embedding.numpy()
get_nn(word_emd,fake_src_embeddings.cpu().numpy(),src_id2word)


0.5965 - same
0.5288 - certain
0.5216 - actual
0.5175 - that
0.5148 - particular


In [21]:
fake_src_embeddings=(mapping(Variable(src_embeddings, requires_grad=False).cuda())).cuda()
tgt_embeddings=(tgt_embeddings).cuda()

In [27]:
fake_src_embeddings=torch.FloatTensor(fake_src_embeddings.cpu().data.numpy()).cuda()
# type(fake_src_embeddings)
# src_embedding=src_embeddings.cpu().detach().numpy()
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2, whiten=True)  # TSNE(n_components=2, n_iter=3000, verbose=2)
# pca.fit(np.vstack([src_embedding, spanish_embeddings]))
# print('Variance explained: %.2f' % pca.explained_variance_ratio_.sum())

torch.cuda.FloatTensor

In [ ]:
# import matplotlib.pyplot as plt


# def plot_similar_word(src_words, src, src_emb, tgt_words, tgt, tgt_emb, pca):

#     Y = []
#     word_labels = []
#     for sw in src_words:
#         Y.append(src_emb[src.get_word_id(sw)])
#         word_labels.append(sw)
#     for tw in tgt_words:
#         Y.append(tgt_emb[tgt.get_word_id(tw)])
#         word_labels.append(tw)

#     # find tsne coords for 2 dimensions
#     Y = pca.transform(Y)
#     x_coords = Y[:, 0]
#     y_coords = Y[:, 1]

#     # display scatter plot
#     plt.figure(figsize=(10, 8), dpi=80)
#     plt.scatter(x_coords, y_coords, marker='x')

#     for k, (label, x, y) in enumerate(zip(word_labels, x_coords, y_coords)):
#         color = 'blue' if k < len(src_words) else 'red'  # src words in blue / tgt words in red
#         plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontsize=19,
#                      color=color, weight='bold')

#     plt.xlim(x_coords.min() - 0.2, x_coords.max() + 0.2)
#     plt.ylim(y_coords.min() - 0.2, y_coords.max() + 0.2)
#     plt.title('Visualization of the multilingual word embedding space')

#     plt.show()
    

In [ ]:
# src_words = ['dog', 'the', 'the', 'tennis', 'cat', 'conference']
# tgt_words = ['perro', 'el', 'la', u'tenis',  'gato', 'conferencia']

# plot_similar_word(src_words,english,english_embeddings.cpu().detach().numpy(),tgt_words,spanish,spanish_embeddings.cpu().detach().numpy(),pca)
# # english.get_word_id

In [28]:
def get_nn_avg_dist(emb, query, knn=10):
    """
    Compute the average distance of the `knn` nearest neighbors
    for a given set of embeddings and queries.
    Use Faiss if available.
    """
    
    bs = 1024
    all_distances = []
    emb = emb.transpose(0, 1).contiguous()
    for i in range(0, query.shape[0], bs):
        distances = query[i:i + bs].mm(emb)
        best_distances, _ = distances.topk(knn, dim=1, largest=True, sorted=True)
        all_distances.append(best_distances.mean(1).cpu())
    all_distances = torch.cat(all_distances)
    return all_distances.numpy()

def get_candidates(emb1,emb2):
    bs = 128

    all_scores = []
    all_targets = []

    # number of source words to consider
    n_src = emb1.size(0)
    n_src =15000
    knn = 10

        # average distances to k nearest neighbors
    average_dist1 = torch.from_numpy(get_nn_avg_dist(emb2, emb1, knn))
    average_dist2 = torch.from_numpy(get_nn_avg_dist(emb1, emb2, knn))
    average_dist1 = average_dist1.type_as(emb1)
    average_dist2 = average_dist2.type_as(emb2)

    # for every source word
    for i in range(0, n_src, bs):

        # compute target words scores
        scores = emb2.mm(emb1[i:min(n_src, i + bs)].transpose(0, 1)).transpose(0, 1)
        scores.mul_(2)
        scores.sub_(average_dist1[i:min(n_src, i + bs)][:, None] + average_dist2[None, :])
        best_scores, best_targets = scores.topk(2, dim=1, largest=True, sorted=True)

        # update scores / potential targets
        all_scores.append(best_scores.cpu())
        all_targets.append(best_targets.cpu())

    all_scores = torch.cat(all_scores, 0)
    all_targets = torch.cat(all_targets, 0)

    all_pairs = torch.cat([torch.arange(0, all_targets.size(0)).long().unsqueeze(1),all_targets[:, 0].unsqueeze(1)], 1)
    assert all_scores.size() == all_pairs.size() == (n_src, 2)

    # sort pairs by score confidence
    diff = all_scores[:, 0] - all_scores[:, 1]
    reordered = diff.sort(0, descending=True)[1]
    all_scores = all_scores[reordered]
    all_pairs = all_pairs[reordered]
    
    
    selected = all_pairs.max(1)[0] <= 15000
    mask = selected.unsqueeze(1).expand_as(all_scores).clone()
    all_scores = all_scores.masked_select(mask).view(-1, 2)
    all_pairs = all_pairs.masked_select(mask).view(-1, 2)
    return all_pairs


In [29]:

def create_dictionary(src_embedding,tgt_embedding):
    src2tgt = get_candidates(src_embedding,tgt_embedding)
    tgt2src = get_candidates(tgt_embedding,src_embedding)
    
    s2t_candidates = set([(a, b) for a, b in src2tgt])
    t2s_candidates = set([(a, b) for a, b in tgt2src])
#     print(s2t_candidates)
    final_pairs = s2t_candidates & t2s_candidates
    if len(final_pairs) == 0:
        print("warning")
        return None
    dico = torch.LongTensor(list([[a, b] for (a, b) in final_pairs]))

    print('New train dictionary of %i pairs.' % dico.size(0))
    return dico.cuda()
    
        
    

In [31]:
dico=create_dictionary(fake_src_embeddings,tgt_embeddings)
# print(type(src_embedding))

New train dictionary of 2 pairs.


In [33]:
print(dico)



 14940  12634
 12634  14940
[torch.cuda.LongTensor of size 2x2 (GPU 0)]



In [7]:
def dist_mean_cosine(src_emb, tgt_emb):
        """
        Mean-cosine model selection criterion.
        """
        
        # get normalized embeddings
        src_emb = mapping(src_emb)
        tgt_emb = tgt_emb
        src_emb = src_emb / src_emb.norm(2, 1, keepdim=True).expand_as(src_emb)
        tgt_emb = tgt_emb / tgt_emb.norm(2, 1, keepdim=True).expand_as(tgt_emb)
        

        s2t_candidates = get_candidates(src_emb, tgt_emb, _params)
        t2s_candidates = get_candidates(tgt_emb, src_emb, _params)
        dico = build_dictionary(src_emb, tgt_emb, _params, s2t_candidates, t2s_candidates)
        dico_max_size = 10000

      
        if dico is None:
            mean_cosine = -1e9
        else:
            mean_cosine = (src_emb[dico[:dico_max_size, 0]] * tgt_emb[dico[:dico_max_size, 1]]).sum(1).mean()
            print("Mean cosine", mean_cosine)
            